<a href="https://colab.research.google.com/github/tuyishimejohnson/auto-encoders/blob/main/Johnson_Tuyishime_Network_Anomaly_Detection_using_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# Network Anomaly Detection using Autoencoders

- Ananth Sankar, Solutions Architect at NVIDIA.
- Eric Harper, Solutions Architect, Global Telecoms at NVIDIA.

Welcome to the second lab of this series!

In the previous lab we used XGBoost, a powerful and efficient tree-based algorithm for classification of anomalies. We were able to almost perfectly identify the anomalous data in the KDD99 dataset and which type of anomaly occurred.  However, in the real-world, *labeled* data can be expensive and hard to come by. Especially with network security, zero-day attacks can be the most challenging and also the most important attacks to detect. Since, by definition, these attacks are happening for the first time, there will be no way to have labels from them.

So how do we approach *this* problem?

For starters, we could have security analysts investigate the network packets and label anomalous ones. But that solution doesn't scale and our models might have difficulty identifying attacks that haven't occurred before.

Our solution *needs to use* "unsupervised learning." Unsupervised learning is the class of machine learning and deep learning algorithms that enable us to draw inferences from our dataset without labels.



In this lab we will use autoencoders (AEs) to detect anomalies in the KDD99 dataset. There are a lot of advantages to using autoencoders for detecting anomalies. One main advantage is the that AEs can learn non-linear relationships in the data.

While we will not be using the labels in the KDD99 dataset explicitly for model training, we will be using them to evaluate how well our model is doing at detecting the anomalies.  We will also use the labels to see if the AE is embedding the anomalies in latent space according to the type of anomaly.

Note that we will be using Keras as the deep learning framework for this lab. Keras is an open source neural network library written in Python and it is designed to enable fast experimentation with deep neural networks.

In [1]:
# Import libraries that will be needed for the lab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import os, datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import plot_model
%load_ext tensorboard

import pickle

import random
random.seed(123)

data_path = './data'

## Section 1: Dataset

Let's load the preprocessed train and test data that we created in Lab 1. We'll think through the same research question we did in Lab 1 again since it helps us to understand exactly *how* our model is learning.

*How does the ratio of anomalies to normal data impact results and why?*


Recall that when using XGBoost, the ratio didn't impact training meaningfully. Anomalies were simply *a class* of our dataset, not made special in any way by their rare nature. Using AutoEncoders, you'll see that that's no longer true. We'll explore the questions of *how rare is rare enough?* and *how does that impact our ability to identify multiple classes of anomalies?*.

In the cell below, choose to either use 1% or 5% anomaly in your dataset by setting the <code>pct_anomalies</code> parameter to .01 or .05 respectively. If you are taking this in an in-person workshop, choose a partner and do both so you can compare and contrast.

In [ ]:
pct_anomalies = .01

In [ ]:
!python preprocess_data.py --pct_anomalies $pct_anomalies

In [ ]:
filename = './preprocessed_data_full.pkl'
input_file = open(filename,'rb')
preprocessed_data = pickle.load(input_file)
input_file.close()

Recall that we have constructed train and test sets where we removed most of anomalous data in the KDD99 dataset.  This lets us simulate a more realistic anomaly detection problem where anomalies only comprise a small percentage of the data.  We also trained a label encoder on the anomalous labels.  This will allow us to go back and forth between labels and their encoded values.

In [ ]:
for key in preprocessed_data:
    print(key)

In [ ]:
le = preprocessed_data['le']
x_train = preprocessed_data['x_train']
y_train = preprocessed_data['y_train']
x_test = preprocessed_data['x_test']
y_test = preprocessed_data['y_test']

### 1.1 Data Preprocessing

Most of the data preprocessing has already been done in Lab 1. We one-hot encoded the categorical variables and separated the labels off from the input data. After this the data was ready for the XGBoost model.  For training deep autoencoder models, the input data will also have to be scaled between 0 and 1.

In [ ]:
# Normalize the testing and training data using the MinMaxScaler from the scikit learn package
scaler = MinMaxScaler()

# Make sure to only fit the scaler on the training data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# convert the data to FP32
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

## Section 2: Deep Autoencoders

Autoencoders are a subset of neural network architectures where the output dimension is the same as the input dimension. Autoencoders have two networks, an encoder and a decoder.  The encoder encodes its input data into a smaller dimensional space, called the latent space.  The decoder network tries to reconstruct the original data from the latent encoding. Typically, the encoder and decoder are symmetric, and the latent space is a bottleneck. The autoencoder has to learn essential characteristics of the data to be able to do a high-quality reconstruction of the data during decode.

![image.png](attachment:image.png)

In this lab, we'll be using deep autoencoders with dropout. Dropout is a way to control overfitting by randomly
omitting subsets of features at each iteration of a training procedure.

### 2.1 Keras Model Definition

Next we will chose the hyperparameters for the Keras autoencoder model.

- batch_size: this determines how many datapoints we use for each gradient update. Choosing a large batch size will make the model train faster but it might not result in the best accuracy or generalization.

- latent_dim: this determines the size of our bottleneck. Higher values add network capacity while lower values increase the efficiency of the encoding.

- max_epochs: should be high enough for the network to learn from the data, but not so high as to overfit the training data or diverge to a worse result

In [ ]:
input_dim = x_train.shape[1]

# model hyperparameters
batch_size = 512

latent_dim = 4

max_epochs = 10

### 2.2 Encoder Network

Here we define the encoder network.

In [ ]:
# The encoder will consist of a number of dense layers that decrease in size
# as we taper down towards the bottleneck of the network, the latent space
input_data = Input(shape=(input_dim,), name='encoder_input')

# hidden layers
encoder = Dense(96,activation='tanh', name='encoder_1')(input_data)
encoder = Dropout(.1)(encoder)
encoder = Dense(64,activation='tanh', name='encoder_2')(encoder)
encoder = Dropout(.1)(encoder)
encoder = Dense(48,activation='tanh', name='encoder_3')(encoder)
encoder = Dropout(.1)(encoder)
encoder = Dense(16,activation='tanh', name='encoder_4')(encoder)
encoder = Dropout(.1)(encoder)

# bottleneck layer
latent_encoding = Dense(latent_dim, activation='linear', name='latent_encoding')(encoder)

Here we instantiate the encoder model, look at a summary of its layers, and then visualize it.

In [ ]:
encoder_model = Model(input_data, latent_encoding)

encoder_model.summary()

In [ ]:
plot_model(
    encoder_model,
    to_file='encoder_model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB' # TB for top to bottom, LR for left to right
)

Image(filename='encoder_model.png')

### 2.3 Decoder Network [Exercise]

The decoder network comes after the bottle neck of the auto encoder. It takes latent values as input and then outputs reconstructions of the original input data.

**Exercise: Use the fact that the decoder is a mirror image of the encoder network to replace the ##FIXME##s below and the fact that the input of each layer should be the output of the previous layer to replace the ##Replace with... statements.**

In [ ]:
# The decoder network is a mirror image of the encoder network
decoder = Dense(##FIXME##, activation='tanh', name='decoder_1')(##Replace with the layer name that feeds this one##)
decoder = Dropout(.1)(decoder)
decoder = Dense(##FIXME##, activation='tanh', name='decoder_2')(##Replace with the layer name that feeds this one##)
decoder = Dropout(.1)(decoder)
decoder = Dense(##FIXME##, activation='tanh', name='decoder_3')(##Replace with the layer name that feeds this one##)
decoder = Dropout(.1)(decoder)
decoder = Dense(##FIXME##, activation='tanh', name='decoder_4')(##Replace with the layer name that feeds this one##)
decoder = Dropout(.1)(decoder)

# The output is the same dimension as the input data we are reconstructing
reconstructed_data = Dense(input_dim, activation='linear', name='reconstructed_data')(##Replace with the layer name that feeds this one##)

We instantiate the autoencoder model, look at a summary of it's layers, and visualize it.

In [ ]:
autoencoder_model = Model(input_data, reconstructed_data)

autoencoder_model.summary()

In [ ]:
plot_model(
    autoencoder_model,
    to_file='autoencoder_model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB' # TB for top to bottom, LR for left to right
)

Image(filename='autoencoder_model.png')

### 2.4 Compile the Model

Here we specify the loss function [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error) and the optimizer [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam).

In [ ]:
opt = optimizers.Adam(learning_rate=.00001)

autoencoder_model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])

### 2.5 Train on the KDD99 Data

Let us look at the feature and label columns of our training set before we start training our XG Boost model. Notice we are not using any labels, `y_train` or `y_test`.

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,profile_batch=0,update_freq='epoch',histogram_freq=1)

train_history = autoencoder_model.fit(x_train, x_train,
        shuffle=True,
        epochs=max_epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test),
        callbacks=[tensorboard_callback])

Let's inspect the loss on the train and validation sets.  You should see the loss on the training data and the loss on the validation data converging towards zero.  Notice that the training loss is actually higher than the validation loss.  That's because when we train the network, we are using dropout, which again, "is a way to control overfitting by randomly omitting subsets of features at each iteration of a training procedure." When we validate, we remove the dropout, which gives our network its full strength.

In [ ]:
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.legend(['loss on train data', 'loss on validation data'])

### 2.6 Inspect the TensorBoard

In [ ]:
%tensorboard --logdir logs

## Section 3: Anomaly Detection

In order to detect anomalies, we need to do the following:

1. Reconstruct the test data.
2. Compute the reconstruction scores using MSE.
3. Set a threshold to label test datapoints as anomalies.

In [ ]:
# Reconstruct the data using our trainined autoencoder model.
x_test_recon = autoencoder_model.predict(x_test)

# the reconstruction score is the mean of the reconstruction errors (relatively high scores are anomalous)
reconstruction_scores = np.mean((x_test - x_test_recon)**2, axis=1)

Let's analyze the reconstruction scores.

In [ ]:
# store the reconstruction data in a Pandas dataframe
anomaly_data = pd.DataFrame({'recon_score':reconstruction_scores})

# if our reconstruction scores our normally distributed we can use their statistics
anomaly_data.describe()

In [ ]:
# plotting the density will give us an idea of how the reconstruction scores are distributed
plt.xlabel('Reconstruction Score')
anomaly_data['recon_score'].plot.hist(bins=200, range=[-.01, .03])

Now we define a function to convert the labels to binary. Recall that the label encoder from the previous lab encoded normal data as 11. `labels`=11 therefore corresponds to normal data and `labels`!=11 corresponds to anomalous data.

In [ ]:
def convert_label_to_binary(label_encoder, labels):
    normal_idx = np.where(label_encoder.classes_ == 'normal.')[0][0]
    my_labels = labels.copy()
    my_labels[my_labels != normal_idx] = 1
    my_labels[my_labels == normal_idx] = 0
    return my_labels

## Section 4: Model Validation

In [ ]:
# convert our labels to binary
binary_labels = convert_label_to_binary(le, y_test)

# add the binary labels to our anomaly dataframe
anomaly_data['binary_labels'] = binary_labels

# let's check if the reconstruction statistics are different for labeled anomalies
anomaly_data.groupby(by='binary_labels').describe()

We can see from the above that the anomalous data has a mean reconstruction score of </b>0.05</b> while the normal data has a score of <b>~0.004</b>. This is a good sign that our autoencoder has learned to reconstruct normal data but fails to reconstruct anomalous data.

Let's see how our ROC curve looks with the reconstruction score.

In [ ]:
fpr, tpr, thresholds = roc_curve(binary_labels, reconstruction_scores)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='lime', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

The ROC Curve looks great and we have a perfect AUC score.  This is a good sign, but we still need to verify that we can set a threshold which accomplishes our business objective. In this lab we are looking to detect as many anomalies as possible while also minimizing the number of false positives.

### 4.1 Thresholds

There are number of ways to set thresholds.  We give some examples below and we encourage you to experiment by setting your own thresholds and see how they affect the model.

In [ ]:
# We can pick the threshold based on maximizing the true positive rate (tpr)
# and minimizing the false positive rate (fpr)
optimal_threshold_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_threshold_idx]
print(optimal_threshold)

In [ ]:
# Or we assume our reconstructions are normally distributed and label anomalies as those
# that are a number of standard deviations away from the mean
recon_mean = np.mean(reconstruction_scores)
recon_stddev = np.std(reconstruction_scores)

stats_threshold = recon_mean + 5*recon_stddev
print(stats_threshold)

Now, similar to lab1, let us plot the confusion matrix in order to visualize the results.

In [ ]:
# play around here and find the threshold that works for you

#thresh = optimal_threshold
thresh = stats_threshold



print(thresh)

pred_labels = (reconstruction_scores > thresh).astype(int)

results = confusion_matrix(binary_labels, pred_labels)


### 4.2 Confusion Matrix

Note, in the next lab, you will be asked to implement your own confusion matrix since it's a very useful tool we want to draw your attention to. Copy, paste, and store this cell somewhere safe.

In [ ]:
print ('Confusion Matrix: ')

def plot_confusion_matrix(cm, target_names, title='Confusion Matrix', cmap=plt.cm.Greens):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')


plot_confusion_matrix(results, ['Normal','Anomaly'])

Analyzing the confusion matrix, we see that we get optimal results.  The algorithm is labeling most of the normal data anomalous data correctly while mislabeling very infrequently.

Returning to our research question: How does the proportion of anomalies to normal data impact the ability for a Deep Learning based AutoEncoder model to detect them?

## Section 5: Multi-class Classification

This exercise will show how we could potentially create labeled data by using k-means clustering algorithm to group our predicted anomalies in latent space. The goal of this algorithm is to find groups in the data, with the number of groups represented by the variable K. The algorithm works iteratively to assign each data point to one of K groups based on the features that are provided. Data points are clustered based on feature similarity.

For each test datapoint that we've labeled as an anomaly, we will label the clusters based on the label that appears most in the cluster.

In [ ]:
# subset the test data so that we are only looking at the predicted anomalies
x_test_df = pd.DataFrame(x_test)

test_anomalies = x_test_df[pred_labels.astype('bool')]

y_test_df = pd.DataFrame(y_test)
test_anomalies_labels = y_test_df[pred_labels.astype('bool')]

# encode the test anomalies into latent space
encoded_test_anomalies = encoder_model.predict(test_anomalies)

In [ ]:
# apply KMeans to the data in order to create clusters of anomalies
kmeans = KMeans(n_clusters=10, random_state=123)
kmeans.fit(encoded_test_anomalies)

In [ ]:
clusters = pd.DataFrame({'cluster':kmeans.labels_, 'label':test_anomalies_labels[0]})

most_frequent_labels = clusters.groupby('cluster').label.value_counts()

print(most_frequent_labels)

If we replace our clusters with the label that shows up most frequently for each cluster, then we can compare how well the anomalies are clustered together in latent space.

In [ ]:
# this dictionary will map our cluster values to the labels that appear most frequently
cluster_to_label = {}
for cluster in range(0, 10):
    label = most_frequent_labels[cluster].index[0]
    cluster_to_label[cluster] = label

# we then replace the clusters inplace by their label values
clusters.cluster.replace(cluster_to_label, inplace=True)

Now we can plot the confusion matrix to see how well our clusters agree with the labels

In [ ]:
target_names = np.unique(list(clusters.label))
cm = confusion_matrix(clusters.label, clusters.cluster)

print ('Confusion Matrix :')

def plot_confusion_matrix(cm,target_names, title='Confusion matrix', cmap=plt.cm.Greens):
    plt.figure(figsize=(10,10),)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plot_confusion_matrix(cm,target_names)

## Conclusions

- We were successfully able to detect KDD99 anomalies without using the labels for training by using a deep autoencoder.
- We then showed that autoencoder model was embedding the anomalous datapoints in latent space according to the type of anomaly.
- This means we could begin labeling the anomalous data by having SMEs look at a small fraction of the total data.

## References

1. Haowen, Chen, Zhao, Li, Zeyan, Zhihan, . . . Honglin. (2018, February 12). [_Unsupervised Anomaly Detection via Variational Auto-Encoder for Seasonal KPIs in Web Applications_](https://arxiv.org/abs/1802.03903).
1. Ellison, D. (n.d.). [_Fraud Detection Using Autoencoders in Keras with a TensorFlow Backend_](https://www.datascience.com/blog/fraud-detection-with-tensorflow).
1. Sadanand Singh (n.d.). _A Practical guide to Autoencoders_. Previously retrieved from https://sadanand-singh.github.io/posts/autoencoders/ but no longer online as of November 2023.
1. Jeremy Jordan (n.d.). [_Variational Autoencoders_](https://www.jeremyjordan.me/variational-autoencoders/).

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>